In [42]:
# Importing the libraries
import nltk
import torch
import torch.nn as nn
from torch.nn.functional import embedding
from torch.utils.data import DataLoader

In [36]:
max_input_len = 30
max_output_len = 48
sos_token = 0
eos_token = 1
pad_token = 2
batch_size = 16

In [3]:
with open(r"..\dataset\ELRC-417-Swedish_Work_Environ.en-fr.en", 'r', encoding='utf-8') as file:
    lines = file.readlines()
src_sentences = []
#print(len(lines))
for line in lines[1:-1]:
    src_sentences.append(line.lower().strip())

with open(r"..\dataset\ELRC-417-Swedish_Work_Environ.en-fr.fr", 'r', encoding='utf-8') as file:
    lines = file.readlines()
trg_sentences = []
for line in lines[1:-1]:
    trg_sentences.append(line.lower().strip())

In [4]:
nltk.word_tokenize(src_sentences[0])
max(len(nltk.word_tokenize(sentence)) for sentence in trg_sentences)

40

In [5]:
'''
src_word_list = []
for sentence in src_sentences:
    src_word_list.extend([word.lower() for word in sentence[:-1].split()])
#len(src_word_list)
src_vocab = set(src_word_list)

src_vocab = list(src_vocab)
src_vocab.extend(['<unk>','<pad>','<sos>','<eos>','.',','])
trg_word_list = []
for sentence in trg_sentences:
    trg_word_list.extend([word.lower() for word in sentence[:-1].split()])
trg_vocab = list(set(trg_word_list))
trg_vocab.extend(['<unk>','<pad>','<sos>','<eos>','.',','])
trg_vocab
'''

"\nsrc_word_list = []\nfor sentence in src_sentences:\n    src_word_list.extend([word.lower() for word in sentence[:-1].split()])\n#len(src_word_list)\nsrc_vocab = set(src_word_list)\n\nsrc_vocab = list(src_vocab)\nsrc_vocab.extend(['<unk>','<pad>','<sos>','<eos>','.',','])\ntrg_word_list = []\nfor sentence in trg_sentences:\n    trg_word_list.extend([word.lower() for word in sentence[:-1].split()])\ntrg_vocab = list(set(trg_word_list))\ntrg_vocab.extend(['<unk>','<pad>','<sos>','<eos>','.',','])\ntrg_vocab\n"

maybe try nltk.tokenize tomorrow

In [6]:
'''
src_word_to_index = {w:i for i,w in enumerate(src_vocab)}
src_index_to_word = {i:w for i,w in enumerate(src_vocab)}
trg_word_to_index = {w:i for i,w in enumerate(trg_vocab)}
trg_index_to_word = {i:w for i,w in enumerate(trg_vocab)}
'''
class Vocab():
    def __init__(self):
         self.word_to_index = {'<sos>':0,'<eos>':1,'<pad>':2}
         self.index_to_word = {0:'<sos>',1:'<eos>',2:'<pad>'}
         self.word_count = 3
    
    def add_word(self,word):
        if word not in self.word_to_index:
            self.word_to_index[word] = self.word_count
            self.index_to_word[self.word_count] = word
            self.word_count += 1
    
    def add_sentence(self,sentence):
        for word in sentence.split():
            self.add_word(word)



In [7]:
src_vocab = Vocab()
for sentence in src_sentences:
    for word in nltk.word_tokenize(sentence):
        src_vocab.add_word(word)
trg_vocab = Vocab()
for sentence in trg_sentences:
    for word in nltk.word_tokenize(sentence):
        trg_vocab.add_word(word)
len(src_vocab.word_to_index),len(trg_vocab.word_to_index)

(114, 120)

In [45]:
def sentencetoindexes(sentence,vocab):
    return [vocab.word_to_index[word] for word in nltk.word_tokenize(sentence)]
sentencetoindexes(src_sentences[0],src_vocab)
def make_data(src_sentences,trg_sentences,src_vocab,trg_vocab):
    src_data = []
    trg_data = []
    for i,(src_sentence,trg_sentence) in enumerate(zip(src_sentences,trg_sentences)):
        src_indexes = sentencetoindexes(src_sentence,src_vocab)
        src_indexed = [sos_token] + src_indexes + [eos_token] + [pad_token]*(max_input_len-len(src_indexes))
        trg_indexes = sentencetoindexes(trg_sentence,trg_vocab)
        trg_indexed = [sos_token] + trg_indexes + [eos_token] + [pad_token]*(max_output_len-len(trg_indexes))
        src_data.append(src_indexed)
        trg_data.append(trg_indexed)
    src_data = torch.LongTensor(src_data)
    trg_data = torch.LongTensor(trg_data)
    return src_data,trg_data
src_data,trg_data = make_data(src_sentences,trg_sentences,src_vocab,trg_vocab)      
src_data.shape,trg_data.shape
src_batch = DataLoader(src_data,batch_size=batch_size)
trg_batch = DataLoader(trg_data,batch_size=batch_size)
for data in src_batch:
    print(data.shape)
    break

torch.Size([10, 32])


In [44]:
class Encoder(nn.Module):
    def __init__(self,input_dim,embedding_dim,hidden_dim):
        super(Encoder,self).__init__()
        self.embedding = nn.Embedding(input_dim,embedding_dim) 
        self.rnn = nn.GRU(embedding_dim,hidden_dim)
    
    def forward(self,src):
        embedding = self.embedding(src)
        outputs,hidden = self.rnn(embedding)
        return hidden

class Decoder(nn.Module):
    def __init__(self,output_dim,embedding_dim,hidden_dim):
        super(Decoder,self).__init__()
        self.embedding = nn.Embedding(output_dim,embedding_dim)
        self.rnn = nn.GRU(embedding_dim,hidden_dim)
        self.fc = nn.Linear(hidden_dim,output_dim)
    
    def forward_cell(self,input,hidden):
        input = input.unsqueeze(0) #   (1,batch_size)
        embedding = self.embedding(input) # (batch_size,1,embedding_dim)
        output,hidden = self.rnn(embedding,hidden)
        prediction = self.fc(output)
        return prediction,hidden
    def forward(self,x,hidden):
        batch_size = x.shape[0]
        outputs = torch.zeros(max_output_len,batch_size,output_dim)
        input = x[:,0] # (batch_size)
        for i in range(max_output_len):
            output,hidden = self.forward_cell(input,hidden) # (batch_size,output_dim)
            outputs[i] = output
            input = torch.argmax(output,dim=1) # (batch_size)
        return outputs

class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder):
        super(Seq2Seq,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    def forward(self,src,trg):
        hidden = self.encoder(src)
        outputs = self.decoder(trg,hidden)
        return outputs

model = Seq2Seq(Encoder(len(src_vocab.word_to_index),256,512),Decoder(len(trg_vocab.word_to_index),256,512))
model(src_batch,trg_batch).shape
        
            
            
            
            

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(114, 256)
    (rnn): GRU(256, 512)
  )
  (decoder): Decoder(
    (embedding): Embedding(120, 256)
    (rnn): GRU(256, 512)
    (fc): Linear(in_features=512, out_features=120, bias=True)
  )
)

torch.Size([4])